In [2]:
import os
import cv2
import pydub
import math
import numpy as np
import matplotlib.pyplot as plt
import pickle as p

In [3]:
#define segment size

segment_size = 30
segment_size = segment_size*1000

In [ ]:
#split all songs into parts and save
songs = os.listdir("Songs/")
for song in songs:
    d = pydub.AudioSegment.from_mp3("Songs/" + song)
    current_loc = 0
    for i in range(math.ceil(len(d)/segment_size)):
        if not current_loc+segment_size > len(d):
            e = d[current_loc:current_loc+segment_size]
            #get next [segment size] of music
        else:
            #get last parts of music and fill in the rest of the 30 seconds with silence
            e = d[current_loc:]
            silence = pydub.AudioSegment.silent(duration=(segment_size-(len(d)-current_loc)))
            e = e + silence
        current_loc+=segment_size
        e.export("segments/Segment_{0}_of_{1}.wav".format(i,song), format="wav")


In [ ]:
#use arss to convert audio to images
segments = os.listdir("segments/")
for segment in segments:
    seg_path = "segments/" + segment
    seg_path = seg_path.replace(" ","\\ ")
    out_path = "specs/" + segment + ".bmp"
    out_path = out_path.replace(" ","\\ ")
    os.system("./arss {0}  {1} -q -a -min 20 -max 15000 -p 15 -b 24".format(seg_path, out_path))


In [ ]:
#prepare data for neural network
specs = sorted(os.listdir("specs/"))
full_specs = []
print(specs[0])
for i, spec in enumerate(specs):
    if i == 0:
        full_specs = cv2.rotate(cv2.imread("specs/" + spec, cv2.IMREAD_GRAYSCALE),cv2.ROTATE_90_CLOCKWISE)
    else:
        img = cv2.rotate(cv2.imread("specs/" + spec, cv2.IMREAD_GRAYSCALE),cv2.ROTATE_90_CLOCKWISE)
        full_specs=np.concatenate((full_specs,img), axis=0)
print(full_specs.shape)

x=[]
context_len = 20
for i in range(1,len(full_specs)-context_len):
    x.append(full_specs[i:i+context_len])
x=np.array(x)
x = x/255.0

print(x.shape)
y = full_specs[context_len:-1]/255.0



In [ ]:
#visualize
plt.figure()
plt.axis("off")
plt.margins(0)
i = 0

plt.imshow(x[i], cmap="gray")

plt.show()

#prepare for arss
#cv2.imwrite("spec.bmp",cv2.rotate(x[i]*255.0, cv2.ROTATE_90_COUNTERCLOCKWISE))
#os.chdir(os.path.abspath("."))
#os.system("ffmpeg -i spec.png -pix_fmt rgb24  spec.bmp -y -hide_banner")

In [5]:
#save to pickle
xp = open("X.pickle","wb")
yp = open("Y.pickle","wb")
p.dump(x,xp)
p.dump(y,yp)
xp.close()
yp.close()